# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: GPT-2
* Evaluation approach: HuggingFace Trainer
* Fine-tuning dataset: sms_spam

## Loading a Quantized Foundation Model

In [1]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import load_dataset
import numpy as np

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("gpt2")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
ds = load_dataset("sms_spam")
ds = ds["train"].train_test_split(test_size=0.2)

ds["train"] = ds["train"]
ds["test"] = ds["test"])

In [3]:
splits = ["train", "test"]

tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = ds[split].map(
        lambda x: tokenizer(x["sms"], truncation=True, padding=True), batched=True
    )

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [4]:
model.config.pad_token_id = model.config.eos_token_id


def compute_metrics(eval_prediction):
    predictions, labels = eval_prediction
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

training_args = TrainingArguments(
        output_dir=".data/sms_classifier",
        learning_rate=0.00002,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True
)

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [6]:


target_modules=["c_attn", "c_proj", "c_fc"]

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    target_modules=target_modules
)

model = prepare_model_for_kbit_training(model)
print(model)
lora_model = get_peft_model(model, lora_config)

lora_model.print_trainable_parameters()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear4bit(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, ele

In [7]:
training_args.output_dir =".data/sms_classifier_lora"

train_lora = tokenized_dataset['train'].rename_column('label', 'labels')
test_lora = tokenized_dataset['test'].rename_column('label', 'labels')
    
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.532000,0.690224,0.855605


Checkpoint destination directory .data/sms_classifier_lora/checkpoint-1115 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1115, training_loss=0.6762004835188656, metrics={'train_runtime': 326.8734, 'train_samples_per_second': 13.641, 'train_steps_per_second': 3.411, 'total_flos': 542723894369280.0, 'train_loss': 0.6762004835188656, 'epoch': 1.0})

In [8]:
lora_model.save_pretrained("gpt-qlora")


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [9]:
from peft import AutoPeftModelForSequenceClassification
loaded_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-qlora")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [12]:
#still researching how to fix this
loaded_model = model.to("cuda")
trainer.model = loaded_model
wlora_results = trainer.evaluate()

You shouldn't move a model when it is dispatched on multiple devices.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.